In [208]:
import os
import re
import ast
import html
import json
from collections import Counter
from html.parser import HTMLParser

In [ ]:
root_path = '/Users/vivianamarquez/Desktop/prisoneducation/raw_html/'

# Get all types

Edit: I might not need this after all... I created a function below to parse the content 

In [129]:
def get_all_types(html_content):
    get_types = html_content.split('"type":')
    all_types = []

    for item in get_types[1:]:
        typ = item.split(",")[0].replace('"',"")
        all_types.append(typ)
    return all_types

In [139]:
all_types = []
all_files = []
error_pages = []

for root, dirs, files in os.walk(root_path):
    for file in files:
        file_path = os.path.join(root, file)
        if file_path.endswith('.html'):
            all_files.append(file_path)
            
            with open(file_path, 'r') as file:
                html_content = file.read()

            if "503 Service Temporarily Unavailable" in html_content:
                error_pages.append(file_path)
            else:
                typs = get_all_types(html_content)
                all_types.append(typs)

In [143]:
Counter(sum(all_types,[]))

Counter({'TITLE': 93,
         '': 14,
         'IMAGE': 14,
         'TEXT': 89,
         'SHORT_ANSWER': 14,
         'MULTIPLE_CHOICE': 34})

# Get one of each type

In [160]:
[sorted(i) for i in set([tuple(set(t)) for t in all_types]) if len(i) >= 5]

[['', 'IMAGE', 'SHORT_ANSWER', 'TEXT', 'TITLE'],
 ['', 'IMAGE', 'MULTIPLE_CHOICE', 'TEXT', 'TITLE']]

In [162]:
[sorted(tuple(set(t))) for t in all_types].index(['', 'IMAGE', 'SHORT_ANSWER', 'TEXT', 'TITLE'])

0

In [166]:
all_files[0]

'/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_12/page_1.html'

In [164]:
[sorted(tuple(set(t))) for t in all_types].index(['', 'IMAGE', 'MULTIPLE_CHOICE', 'TEXT', 'TITLE'])

14

In [167]:
all_files[14]

'/Users/vivianamarquez/Desktop/prisoneducation/raw_html/lesson_13/page_1.html'

In [169]:
path = all_files[0]
with open(path, 'r') as file:
    html_content = file.read()

In [171]:
all_types[0]

['TITLE', '', 'IMAGE', 'TEXT', 'SHORT_ANSWER', 'TITLE', 'TEXT']

In [190]:
def parse_nested_braces(string):
    stack = []
    results = []
    current = []

    for char in string:
        if char == '{':
            stack.append('{')
            current.append(char)
        elif char == '}' and stack:
            stack.pop()
            current.append(char)
            if not stack:  # Completed a nested structure
                results.append(''.join(current))
                current = []
        elif stack:
            current.append(char)

    return results

In [228]:
def parse_key_value_pairs(string):
    def is_key_start(index):
        # Check if the current index is the start of a key
        return string[index] == '"' and (index == 0 or string[index - 1] not in '\\')

    def find_key_end(start_index):
        # Find the end of the key (next unescaped quote)
        index = start_index + 1
        while index < len(string):
            if string[index] == '"' and string[index - 1] != '\\':
                return index
            index += 1
        return -1

    def find_value(index):
        # Find the end of the value, handling nested objects and arrays
        if string[index] in '{[':
            # Handle nested structure
            opening = string[index]
            closing = '}' if opening == '{' else ']'
            depth = 1
            index += 1
            while index < len(string) and depth > 0:
                if string[index] == opening:
                    depth += 1
                elif string[index] == closing:
                    depth -= 1
                index += 1
            return index
        else:
            # Handle simple value
            while index < len(string) and string[index] not in ',}':
                index += 1
            return index

    results = {}
    i = 0
    while i < len(string):
        if is_key_start(i):
            key_end = find_key_end(i)
            if key_end != -1:
                key = string[i + 1:key_end]
                # Skip to the colon and the following whitespace
                i = key_end + 2
                while i < len(string) and string[i] in ' \t':
                    i += 1
                value_end = find_value(i)
                value = string[i:value_end].strip()
                results[key] = value
                i = value_end
        i += 1

    return results

In [224]:
parse_nested_braces(html_content[42:-1])[2]

'{"frozen":false,"userFile":null,"pageItemAttributes":null,"answers":[],"pageId":283673,"isScored":null,"pageItemPrecondition":[],"formulas":[],"explanation":null,"attrs":{},"scoreType":"NOT_SCORED","minWordLength":null,"index":2,"optional":false,"name":"Text","value":"\n<p>\n <span &quot;space="" 15pt;="" 163);="" 700;="" 83,="" baseline;="" color:="" font-family:="" font-size:="" font-variant-alternates:="" font-variant-east-asian:="" font-variant-numeric:="" font-weight:="" mono&quot;,="" monospace;="" normal;="" preserve;\\"="" rgb(0,="" style=\'\\"background-color:\' transparent;="" vertical-align:="" white-space-collapse:="">\n  Goal for this session:\n </span>\n <span &quot;space="" 15pt;="" baseline;="" font-family:="" font-size:="" font-variant-alternates:="" font-variant-east-asian:="" font-variant-numeric:="" mono&quot;,="" monospace;="" normal;="" preserve;\\"="" style=\'\\"background-color:\' transparent;="" vertical-align:="" white-space-collapse:="">\n  better understand

In [235]:
parse_key_value_pairs(parse_nested_braces(html_content[42:-1])[1])

{'frozen': 'false',
 'userFile': '{"frozen":false,"contentPath":"/media/430/9646/6990b704-9bd2-4faa-9619-12fb2e8e2a8b-Untitled.png","wordCount":null,"pageCount":null,"partialPath":"/430/9646/6990b704-9bd2-4faa-9619-12fb2e8e2a8b-Untitled.png","video":false,"audio":false,"timeLength":null,"pdf":false,"image":true,"parentId":623650,"mediaPath":"/media/430/9646/6990b704-9bd2-4faa-9619-12fb2e8e2a8b-Untitled.png","fileType":"image","type":"","fileName":"Untitled.png","extension":"png","description":null,"longId":279650,"transcribedClosedCaption":{"present":false},"id":279650,"modified":false,"idName":"id","compositeKeys":null,"valid":true,"new":false}',
 'pageItemAttributes': 'null',
 'answers': '[]',
 'pageId': '283673',
 'isScored': 'null',
 'pageItemPrecondition': '[]',
 'formulas': '[]',
 'explanation': 'null',
 'attrs': '{}',
 'scoreType': '"NOT_SCORED"',
 'minWordLength': 'null',
 'index': '1',
 'optional': 'false',
 'name': '"Image"',
 'value': 'null',
 'type': '"IMAGE"',
 'id': '6236

In [203]:
parse_dict_with_html(parse_nested_braces(html_content[42:-1])[0])

{}

In [205]:
json.loads(parse_nested_braces(html_content[42:-1])[2])

JSONDecodeError: Invalid control character at: line 1 column 270 (char 269)

In [198]:
parse_nested_braces(html_content[42:-1])[2]

'{"frozen":false,"userFile":null,"pageItemAttributes":null,"answers":[],"pageId":283673,"isScored":null,"pageItemPrecondition":[],"formulas":[],"explanation":null,"attrs":{},"scoreType":"NOT_SCORED","minWordLength":null,"index":2,"optional":false,"name":"Text","value":"\n<p>\n <span &quot;space="" 15pt;="" 163);="" 700;="" 83,="" baseline;="" color:="" font-family:="" font-size:="" font-variant-alternates:="" font-variant-east-asian:="" font-variant-numeric:="" font-weight:="" mono&quot;,="" monospace;="" normal;="" preserve;\\"="" rgb(0,="" style=\'\\"background-color:\' transparent;="" vertical-align:="" white-space-collapse:="">\n  Goal for this session:\n </span>\n <span &quot;space="" 15pt;="" baseline;="" font-family:="" font-size:="" font-variant-alternates:="" font-variant-east-asian:="" font-variant-numeric:="" mono&quot;,="" monospace;="" normal;="" preserve;\\"="" style=\'\\"background-color:\' transparent;="" vertical-align:="" white-space-collapse:="">\n  better understand

In [180]:
html_content.split('"type":')[0]

'{"preconditionCandidates":[],"pageItems":[{"frozen":false,"userFile":null,"pageItemAttributes":null,"answers":[],"pageId":283673,"isScored":null,"pageItemPrecondition":[],"formulas":[],"explanation":null,"attrs":{},"scoreType":"NOT_SCORED","minWordLength":null,"index":0,"optional":false,"name":"Title","value":"Lesson #12: Flask",'

In [177]:
html_content.split('"type":')[1]

'"TITLE","id":623649,"modified":false,"longId":623649,"idName":"id","compositeKeys":null,"valid":true,"new":false},{"frozen":false,"userFile":{"frozen":false,"contentPath":"/media/430/9646/6990b704-9bd2-4faa-9619-12fb2e8e2a8b-Untitled.png","wordCount":null,"pageCount":null,"partialPath":"/430/9646/6990b704-9bd2-4faa-9619-12fb2e8e2a8b-Untitled.png","video":false,"audio":false,"timeLength":null,"pdf":false,"image":true,"parentId":623650,"mediaPath":"/media/430/9646/6990b704-9bd2-4faa-9619-12fb2e8e2a8b-Untitled.png","fileType":"image",'

In [178]:
html_content.split('"type":')[2]

'"","fileName":"Untitled.png","extension":"png","description":null,"longId":279650,"transcribedClosedCaption":{"present":false},"id":279650,"modified":false,"idName":"id","compositeKeys":null,"valid":true,"new":false},"pageItemAttributes":null,"answers":[],"pageId":283673,"isScored":null,"pageItemPrecondition":[],"formulas":[],"explanation":null,"attrs":{},"scoreType":"NOT_SCORED","minWordLength":null,"index":1,"optional":false,"name":"Image","value":null,'

In [179]:
html_content.split('"type":')[3]

'"IMAGE","id":623650,"modified":false,"longId":623650,"idName":"id","compositeKeys":null,"valid":true,"new":false},{"frozen":false,"userFile":null,"pageItemAttributes":null,"answers":[],"pageId":283673,"isScored":null,"pageItemPrecondition":[],"formulas":[],"explanation":null,"attrs":{},"scoreType":"NOT_SCORED","minWordLength":null,"index":2,"optional":false,"name":"Text","value":"\n<p>\n <span &quot;space="" 15pt;="" 163);="" 700;="" 83,="" baseline;="" color:="" font-family:="" font-size:="" font-variant-alternates:="" font-variant-east-asian:="" font-variant-numeric:="" font-weight:="" mono&quot;,="" monospace;="" normal;="" preserve;\\"="" rgb(0,="" style=\'\\"background-color:\' transparent;="" vertical-align:="" white-space-collapse:="">\n  Goal for this session:\n </span>\n <span &quot;space="" 15pt;="" baseline;="" font-family:="" font-size:="" font-variant-alternates:="" font-variant-east-asian:="" font-variant-numeric:="" mono&quot;,="" monospace;="" normal;="" preserve;\\"="